## Import Libraries

In [1]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
from pydgilib_extra import *
from atprogram.atprogram import atprogram

In [4]:
from os import getcwd, path, pardir

## Compile and program project

In [5]:
project_path = path.join(getcwd(), "FFT-S")
project_path

'C:\\Users\\erikw_000\\Documents\\GitHub\\Atmel-SAML11\\Experiments\\Workloads\\FFT\\FFT-S'

In [6]:
atprogram(project_path, verbose=2)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:21190
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00000000. Size = 0x00005810.
[DEBUG] Memory segment base written at 0x00005810. Size = 0x00000068.
[DEBUG] Memory segment base written at 0x0000fb00. Size = 0x00000020.
[DEBUG] Memory segment base written at 0x00804000. Size = 0x00000020.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.



0

## Data Logging

In [7]:
live_plot = True

Create a figure for the plot.

In [8]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [9]:
if live_plot:
    config_dict = {
        "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
        "plot_pins": [False, False, True, True],
        "gpio_delay_time" : 0.0007,
        "plot_pins_method": "line",
        "plot_xmax": 6,
        "fig": fig,
        "window_title": "Experiment FFT",
        "file_name_base": "experiment_fft"
    }
else:
    config_dict = {
        "loggers": [LOGGER_OBJECT, LOGGER_CSV],
        "gpio_delay_time" : 0.0007,
        "file_name_base": "experiment_fft"
    }

Stop criteria to pass to the logger:

In [10]:
def stop_fn(logger_data):
    return all(logger_data.gpio.values[-1])

Stop criteria to pass to the parser:

In [11]:
def stop_function(pin_values):
    return all(pin_values)

In [12]:
repetitions = 10

Perform the measurement.

In [ ]:
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
        
charges_allocate = []
charges_fft = []
charges_switch_to_ifft = []
charges_ifft = []
charges_free = []
times_allocate = []
times_fft = []
times_switch_to_ifft = []
times_ifft = []
times_free = []


with DGILibExtra(**config_dict) as dgilib:
    for _ in range(repetitions):
        dgilib.device_reset()
        dgilib.logger.log(10,stop_fn)
        charge2, time2 = power_and_time_per_pulse(dgilib.data, 2, 0, stop_function=stop_function)
        charge3, time3 = power_and_time_per_pulse(dgilib.data, 2, 0, stop_function=stop_function)
        
        if len(charge2) != 3 or len(time2) != 3 or len(charge3) != 3 or len(time3) != 3:
            print(f"Parsing of measurement data failed, charge2:{charge2}, time2:{time2}, charge3:{charge3}, time3:{time3},")
            break
        charges_allocate.append(charge2[0])
        charges_fft.append(charge3[0])
        charges_switch_to_ifft.append(charge2[1])
        charges_ifft.append(charge3[1])
        charges_free.append(charge2[2])
        times_allocate.append(time2[0])
        times_fft.append(time3[0])
        times_switch_to_ifft.append(time2[1])
        times_ifft.append(time3[1])
        times_free.append(time2[2])
        # dgilib.logger.plotobj.ax.set_title(f"Previous sample: charge: {sum(charge2+charge3)*1e3:.06} mC, time: {sum(time2+time3):.06} s")
        dgilib.empty_data()

    charges = charges_allocate + charges_fft + charges_switch_to_ifft + charges_ifft + charges_free 
    times = times_allocate + times_fft + times_switch_to_ifft + times_ifft + times_free

    dgilib.logger.plotobj.ax.set_title(f"Average of {repetitions} samples: charge: {sum(charges)/len(charges)*1e3:.06} mC, time: {sum(times)/len(times):.06} s")

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'times': times, 'charges':charges})

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
sum(charges)/repetitions

In [ ]:
print("%.0100f" % (sum(charges)/repetitions))